In [ ]:
# Archit Kulkarni, Adhithya Narayanan, Vibhu Ambil, Tyler Youngberg
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
'''
Title: M5 Data Preprocessing
Author: Quinn Wang
Date: March 2020
Availability: https://www.kaggle.com/qcw171717/naive-baseline/
'''

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from tqdm import tqdm
df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')
price_df = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv")
cal_df = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv")
cal_df["d"]=cal_df["d"].apply(lambda x: int(x.split("_")[1]))
price_df["id"] = price_df["item_id"] + "_" + price_df["store_id"] + "_validation"
for i in range(1886, 1914):
    df["F_" + str(i)] = 0

'''end Quinn Wang'''

In [ ]:
import lightgbm as lgb

Data Preprocessing Steps:

1. Convert categorical observations to int format
2. Shorten training data to last 28 days before predictions (days 1858-1886)
3. Create validation data from days 1887-1913

In [ ]:
temp = np.zeros((30490,), dtype=int)
k = 0
for i in df["state_id"]:
    if i == "CA":
        temp[k] = 0
    if i == "TX":
        temp[k] = 1
    if i == "WI":
        temp[k] = 2
    k+=1
df["state_id"] = pd.DataFrame(temp)

In [ ]:
temp = np.zeros((30490,), dtype=int)
k = 0
for i in df["store_id"]:
    if i == "CA_1":
        temp[k] = 0
    elif i == "CA_2":
        temp[k] = 1
    elif i == "CA_3":
        temp[k] = 2
    elif i == "CA_4":
        temp[k] = 3
    elif i == "TX_1":
        temp[k] = 4
    elif i == "TX_2":
        temp[k] = 5
    elif i == "TX_3":
        temp[k] = 6
    elif i == "WI_1":
        temp[k] = 7
    elif i == "WI_2":
        temp[k] = 8
    elif i == "WI_3":
        temp[k] = 9
    k+=1
df["store_id"] = pd.DataFrame(temp)

In [ ]:
temp = np.zeros((30490,), dtype=int)
k = 0
for i in df["cat_id"]:
    if i == "HOBBIES":
        temp[k] = 0
    if i == "FOODS":
        temp[k] = 1
    if i == "HOUSEHOLD":
        temp[k] = 2
    k+=1
df["cat_id"] = pd.DataFrame(temp) 

In [ ]:
temp = np.zeros((30490,), dtype=int)
k = 0
for i in df["dept_id"]:
    if i == "HOBBIES_1":
        temp[k] = 0
    elif i == "HOBBIES_2":
        temp[k] = 1
    elif i == "FOODS_1":
        temp[k] = 2
    elif i == "FOODS_2":
        temp[k] = 3
    elif i == "FOODS_3":
        temp[k] = 4
    elif i == "HOUSEHOLD_1":
        temp[k] = 5
    elif i == "HOUSEHOLD_2":
        temp[k] = 6
    k+=1
df["dept_id"] = pd.DataFrame(temp)

In [ ]:
validation = pd.DataFrame()
train = df
for x in range(1886, 1914):
    s = "d_" + str(x)
    f = "F_" + str(x)
    train = train.drop(columns = [s, f])
    validation[s] = df[s]

In [ ]:
validation

In [ ]:
train = train.drop(columns=["item_id", "id"])

In [ ]:
ids = pd.DataFrame()
ids["dept_id"] = df["dept_id"]
ids["cat_id"] = df["cat_id"]
ids["store_id"] = df["store_id"]
ids["state_id"] = df["state_id"]

In [ ]:
ids

Now, we will be creating a few models to see which one works the best. We will have 5 time periods, the minimum being the last 28 days, and the maximum being the last 800 days.

In [ ]:
train1 = pd.concat([ids, pd.DataFrame(train.values[:, -28:])], axis=1) # Last 28 days
train2 = pd.concat([ids, pd.DataFrame(train.values[:, -50:])], axis=1) # Last 50 days
train3 = pd.concat([ids, pd.DataFrame(train.values[:, -200:])], axis=1) # last 200 days
train4 = pd.concat([ids, pd.DataFrame(train.values[:, -400:])], axis=1) # last 400 days
train5 = pd.concat([ids, pd.DataFrame(train.values[:, -800:])], axis=1) # last 800 days
t = [train1, train2, train3, train4, train5]

To add to the complexity and increase the chance of getting a good model, we will also change the parameters through iteration.

In [ ]:
params_list = [{
   'task': 'train',
   'boosting_type': 'gbdt',
   'objective': 'regression',
   'metric': 'rmse',
   'learning_rate': .1,
    'num_iterations': 75
},
{
   'task': 'train',
   'boosting_type': 'rf'
},
{
    'task':'train',
    'boosting_type': 'dart',
    'metric': 'fair',
    'learning_rate': .1,
    'num_iterations': 75    
},
{
    'task':'train',
    'boosting_type': 'regression',
    'metric': 'rmse',
    'tree_learner': 'feature',
    'num_iterations': 75  
},
{
    'task':'train',
    'boosting_type': 'regression',
    'metric': 'rmse',
    'tree_learner': 'feature',
    'num_iterations': 50  
}]

In [ ]:
train1

In [ ]:
# merge ids dataframe with sales data
pd.concat([ids, train1], axis=1)

In [ ]:
from sklearn.metrics import mean_squared_error as mse
import math
def learner(t_data, v_data, params):
    n = int(t_data.values[0, :].shape[0])
    for x in range(1886, 1914):    
        d = pd.DataFrame()
        
        # setup for training
        t = t_data.values
        v = v_data.values[:, x - 1886]
        lgbm_data = lgb.Dataset(t, label=v, 
                                feature_name=t_data.columns.tolist(), 
                                categorical_feature=['dept_id', 'cat_id', 'store_id', 'state_id'])

        # train data
        gbt = lgb.train(params, lgbm_data) 

        s = str(n) # string name for day in format d_xxxx
        x = pd.DataFrame(gbt.predict(t))

        # add new data to training for next iteration
        t_data[s] = x
        n+=1
        
    forecast = pd.DataFrame(df["id"])
    for i in range(1886, 1914):
        d = "d_" + str(i)
        f = "F" + str(i-1885)
        forecast[f] = train[d]
    print(math.sqrt(mse(v_data.values, forecast.drop(columns=["id"]).values)))
    f2 = forecast.copy()
    f2["id"] = f2["id"].apply(lambda x : x.replace('validation', 'evaluation'))
    forecast = forecast.append(submission_df2).reset_index(drop=True)
    return forecast

In [ ]:
j = 1
for i in range(0, 5):
    params = params_list[i]
    for k in range(0, 5):
        f = learner(train[k], v_data, params)
        name = "learner_" + str(j) + ".csv"
        forecast.to_csv(name, index=False)
        j+=1